In [1]:
import os
from langchain_community.document_loaders import TextLoader
from langchain.memory import ConversationBufferMemory

from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI

os.environ["OPENAI_API_KEY"] = "HIDDEN"

chat = ChatOpenAI(
    openai_api_key=os.environ.get("OPENAI_API_KEY"),
    model='gpt-3.5-turbo'
)


/Users/lpp/Personalized-LLM/env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/lpp/Personalized-LLM/env/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [2]:
from pinecone import Pinecone

pc = Pinecone(api_key='HIDDEN')
	
from pinecone import ServerlessSpec

spec = ServerlessSpec(
    cloud="aws", region="us-west-2"
)

/Users/lpp/Personalized-LLM/env/lib/python3.9/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
from langchain_text_splitters import CharacterTextSplitter


txt_file_path = './static/CSCI2270_Info.txt'
loader = TextLoader(file_path=txt_file_path, encoding="utf-8")
data = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(data)

In [4]:
chunks

[Document(page_content='Summary:\nCSCI2270 is a graduate seminar course that covers the state of the research and practice at the intersection of data management and artificial intelligence. \nWe will explore the symbiotic relationship between AI and effective data management techniques by studying how machine learning techniques can solve database challenges, and the use of data systems to build scalable AI solutions. \nSpecific topics of interest will be the role and use of vector databases and Large Language Models (LLMs) in AI and data management. \nThis class will be structured as a seminar with a semester-long project component. The students will present, review and discuss papers. \nAs groups, they will also conduct a research project over the course of the semester. In addition, there will be several guest lectures by researchers and professional experts in the area.', metadata={'source': './static/CSCI2270_Info.txt'}),
 Document(page_content="Prerequisites and Enrollment:\nCSC

In [5]:
import time

index_name = 'test-rag'
existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]


# connect to index
index = pc.Index(index_name)
time.sleep(1)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [6]:
from langchain.embeddings.openai import OpenAIEmbeddings

embed_model = OpenAIEmbeddings(model="text-embedding-ada-002")
#  We will be using OpenAI's text-embedding-ada-002 model for creating the embeddings, so we set the dimension to 1536.

/Users/lpp/Personalized-LLM/env/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [7]:
# for i, doc in enumerate(chunks):
#     # Extracting text content from each document
#     text_content = doc.page_content
#     # Generate embedding
#     embedding = embed_model.embed_documents(text_content)
#     # Convert embedding to list for Pinecone
#     embedding_list = embedding.tolist()
    
#     # Step 4: Insert into Pinecone
#     index.upsert(vectors={(str(i), embedding_list)})

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [8]:
from langchain_pinecone import PineconeVectorStore
import os
os.environ['PINECONE_API_KEY'] = 'HIDDEN'
docsearch = PineconeVectorStore.from_documents(chunks, embed_model, index_name=index_name)

In [ ]:
# from langchain.vectorstores import Pinecone

# text_field = "text"  # the metadata field that contains our text

# # initialize the vector store object
# vectorstore = Pinecone(
#     index, embed_model.embed_query, text_field
# )

In [9]:
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=chat,
    chain_type="stuff",
    retriever=docsearch.as_retriever(),
    memory=memory
)
query = "What is the course csci2270 by Prof.Ugur Cetintemel about?"
result = conversation_chain({"question": query})
answer = result["answer"]
answer


/Users/lpp/Personalized-LLM/env/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'CSCI2270 is a graduate seminar course at Brown University that focuses on the intersection of data management and artificial intelligence. The course explores how machine learning techniques can solve database challenges and how data systems can be used to build scalable AI solutions. Specific topics include vector databases and Large Language Models (LLMs). The course is structured as a seminar with a semester-long project component, where students will present, review, and discuss papers, as well as conduct a research project. There will also be guest lectures by researchers and experts in the field.'

In [10]:
print(answer)

CSCI2270 is a graduate seminar course at Brown University that focuses on the intersection of data management and artificial intelligence. The course explores how machine learning techniques can solve database challenges and how data systems can be used to build scalable AI solutions. Specific topics include vector databases and Large Language Models (LLMs). The course is structured as a seminar with a semester-long project component, where students will present, review, and discuss papers, as well as conduct a research project. There will also be guest lectures by researchers and experts in the field.


In [ ]:
pc.delete_index(index_name)
